In [ ]:
import cv2
import mediapipe as mp
from dollarpy import Recognizer, Template, Point
import numpy as np
import pickle
import os

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Function to get points from a video
def getPoints(videoURL):
    cap = cv2.VideoCapture(videoURL)
    points = []

    with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
        while cap.isOpened():
            ret, frame = cap.read()
            if ret:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = hands.process(image)

                if results.multi_hand_landmarks:
                    for hand_landmarks in results.multi_hand_landmarks:
                        for landmark in hand_landmarks.landmark:
                            x, y = landmark.x, landmark.y
                            if not (np.isnan(x) or np.isnan(y)):
                                points.append(Point(x, y, stroke_id=0))
            else:
                break

    cap.release()
    return points if points else []

# Function to process multiple videos for a single gesture
def getPointsForMultipleVideos(video_list, label):
    templates = []
    for video in video_list:
        points = getPoints(video)
        if points:
            x_values = [p.x for p in points]
            y_values = [p.y for p in points]
            if max(x_values) - min(x_values) != 0 and max(y_values) - min(y_values) != 0:
                templates.append(Template(label, points))
    return templates

# Function to train templates and save them
def trainTemplates(save_path):
    templates = []

    # Define gestures and their videos
    gesture_videos = {
      
        "right": [
            r"C:\Users\alie0\Downloads\right.mp4",
            r"C:\Users\alie0\Downloads\right1.mp4",
            r"C:\Users\alie0\Downloads\right2.mp4",
            r"C:\Users\alie0\Downloads\right3.mp4",
            r"C:\Users\alie0\Downloads\right4.mp4",
            r"C:\Users\alie0\Downloads\right5.mp4",
            r"C:\Users\alie0\Downloads\right6.mp4",
            r"C:\Users\alie0\Downloads\right7.mp4",
            r"C:\Users\alie0\Downloads\right8.mp4",
            r"C:\Users\alie0\Downloads\right9.mp4",
            r"C:\Users\alie0\Downloads\right10.mp4",
            r"C:\Users\alie0\Downloads\right11.mp4",
            r"C:\Users\alie0\Downloads\Dataset\Dataset\left\right12.mp4",
            r"C:\Users\alie0\Downloads\Dataset\Dataset\left\right13.mp4",
            r"C:\Users\alie0\Downloads\Dataset\Dataset\left\right14.mp4",
            r"C:\Users\alie0\Downloads\Dataset\Dataset\left\right15.mp4",
            r"C:\Users\alie0\Downloads\Dataset\Dataset\left\right16.mp4",
            r"C:\Users\alie0\Downloads\Dataset\Dataset\left\right17.mp4"
        ],
        "left": [
            r"C:\Users\alie0\Downloads\left.mp4",
            r"C:\Users\alie0\Downloads\left1.mp4",
            r"C:\Users\alie0\Downloads\left2.mp4",
            r"C:\Users\alie0\Downloads\left3.mp4",
            r"C:\Users\alie0\Downloads\left4.mp4",
            r"C:\Users\alie0\Downloads\left5.mp4",
            r"C:\Users\alie0\Downloads\left6.mp4",
            r"C:\Users\alie0\Downloads\left7.mp4",
            r"C:\Users\alie0\Downloads\left8.mp4",
            r"C:\Users\alie0\Downloads\left9.mp4",
            r"C:\Users\alie0\Downloads\left10.mp4",
            r"C:\Users\alie0\Downloads\left11.mp4",
            r"C:\Users\alie0\Downloads\left12.mp4",
            r"C:\Users\alie0\Downloads\left13.mp4",
            r"C:\Users\alie0\Downloads\Dataset\Dataset\right\left14.mp4",
            r"C:\Users\alie0\Downloads\Dataset\Dataset\right\left15.mp4",
            r"C:\Users\alie0\Downloads\Dataset\Dataset\right\left16.mp4",
            r"C:\Users\alie0\Downloads\Dataset\Dataset\right\left17.mp4",
            r"C:\Users\alie0\Downloads\Dataset\Dataset\right\left18.mp4",
            r"C:\Users\alie0\Downloads\Dataset\Dataset\right\left19.mp4"
        ]
    }

    # Train templates for each gesture
    for label, videos in gesture_videos.items():
        templates.extend(getPointsForMultipleVideos(videos, label))

    # Save templates to disk
    with open(save_path, 'wb') as f:
        pickle.dump(templates, f)
    print(f"Templates saved to {save_path}")

# Function to load templates from file
def loadTemplates(save_path):
    if os.path.exists(save_path):
        with open(save_path, 'rb') as f:
            return pickle.load(f)
    return []

# Function to get points from a single frame
def getPointsFromFrame(frame):
    points = []

    with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        # Process hand landmarks
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                for landmark in hand_landmarks.landmark:
                    x, y = landmark.x, landmark.y
                    if not (np.isnan(x) or np.isnan(y)):
                        points.append(Point(x, y, stroke_id=0))
    return points

# Main program
template_file = "gesture_templates2.pkl"
if not os.path.exists(template_file):
    trainTemplates(template_file)

templates = loadTemplates(template_file)

# Initialize Recognizer with loaded templates
recognizer = Recognizer(templates)

# Use live camera feed to recognize gestures
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame from camera.")
            break

        points = getPointsFromFrame(frame)
        x_values = [p.x for p in points]
        y_values = [p.y for p in points]

        if points and (max(x_values) - min(x_values) != 0) and (max(y_values) - min(y_values) != 0):
            result = recognizer.recognize(points)

            if result:
                best_match = result[0]
                gesture_name = best_match
                cv2.putText(frame, f"Gesture: {gesture_name}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else:
            cv2.putText(frame, "No gesture detected", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        cv2.imshow("Gesture Recognition", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
